In [1]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib as mlp
# mlp.use("Qt5Agg")
import matplotlib.pyplot as plt
%matplotlib inline
import networkx as nx

import analysis as an
import pandas as pd


import numpy as np
import seaborn as sns

from scipy import stats 
import scipy
import pprint as pp

In [2]:
def zscore(col):
    col_z = (col - col.mean())/col.std(ddof=0)
    return(col_z)

In [3]:
p='/Users/gracer/Google Drive/HCP/HCP_graph/1200/'
atlas=pd.read_csv(os.path.join(p,'brains','atlas.csv'), sep=',')


In [4]:
labels = pd.read_csv('/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/mod_labels.csv', sep=',')

In [5]:
basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/'

In [6]:
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'6_*')
submod_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/6_submod_dict_09-14-2020_04-46-07


In [7]:
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'5_*')
presummary_dict=an.onetoughjar(latest_file)
summary_dict = {}
for key, value in presummary_dict.items():
    print(key)
    summary_dict[key] = {**value[0], **value[1]}


/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/5_summary_dict_09-14-2020_04-30-45
no
ov
ob


In [11]:
summary_dict['no'].keys()

dict_keys(['mean_FC', 'graphs', 'modules', 'corrmat', 'clustering_coeff', 'btn_centrality', 'PC'])

In [9]:
latest_file=an.find_latest(os.path.join(basepath,'tmp'),'7_*')
subgraph_dict=an.onetoughjar(latest_file)

/Users/gracer/Google Drive/HCP/HCP_graph/1200/datasets/tmp/7_subgraph_dict_09-14-2020_04-48-27


In [10]:
subgraph_dict['no'][0].nodes(data=True)

NodeDataView({0: {'clustering': 0.6437710437710438, 'centrality': 0.0064897671177409555, 'PC': 0.7294094283256284, 'sub_modules': 0, 'sub_zDegree': 0.0}, 24: {'clustering': 0.6076388888888888, 'centrality': 0.01252232649388041, 'PC': 0.6808717410992082, 'sub_modules': 1, 'sub_zDegree': 0.0}, 38: {'clustering': 0.5475609756097561, 'centrality': 0.0065647443593222984, 'PC': 0.6353580020933413, 'sub_modules': 2, 'sub_zDegree': 0.0}, 31: {'clustering': 0.6101694915254238, 'centrality': 0.011105955087203292, 'PC': 0.6970049919030915, 'sub_modules': 3, 'sub_zDegree': 0.0}})

In [12]:
for k,v in summary_dict.items():
    # community.induced_graph(partition dictionary, graph)
     comm_graph = community.induced_graph(v['modules']['partition'], v['graphs'])
     v.update(comm_graph = comm_graph)

In [13]:
summary_dict['no'].keys()

dict_keys(['mean_FC', 'graphs', 'modules', 'corrmat', 'clustering_coeff', 'btn_centrality', 'PC', 'comm_graph'])

In [14]:
summary_dict['no']['comm_graph'].nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}})

In [15]:
summary_dict['ob']['comm_graph'].nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}})

In [16]:
summary_dict['ov']['comm_graph'].nodes(data=True)

NodeDataView({0: {}, 1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}})

In [17]:
edges = {}

for group, stuff in summary_dict.items():
    print(group)
    _df = nx.to_pandas_edgelist(stuff['comm_graph'])
    _df.loc[(_df['source'] == _df['target']), 'weight'] = 0

    _df['group']=group
    edges[group]=_df

edge_df=pd.concat(list(edges.values()))

no
ov
ob


In [18]:
edge_df['z_weight']=zscore(edge_df['weight'])

09-14-2020_04-54-19 NumExpr defaulting to 4 threads. 


In [19]:
edge_df.describe()

,source,target,weight,z_weight
count,126.000000,126.000000,126.000000,1.260000e+02
mean,2.571429,5.142857,-2.550741,3.965082e-18
std,2.159365,2.175185,11.558860,1.003992e+00
min,0.000000,0.000000,-82.851796,-6.974876e+00
25%,1.000000,4.000000,-1.060795,1.294154e-01
50%,2.000000,5.500000,0.000000,2.215550e-01
75%,4.000000,7.000000,0.535471,2.680655e-01
max,8.000000,8.000000,10.353384,1.120840e+00


In [20]:
for k,v in summary_dict.items():
    test=edge_df[edge_df['group']==k]
    keyz = list(zip(test['source'],test['target']))
    values=test['z_weight']
    up_dict={}
    for i in range(len(keyz)):
        up_dict[keyz[i]]={'z_edge':values[i]}
    nx.set_edge_attributes(v['comm_graph'], up_dict)


In [21]:
aes_dict={'no':{},
          'ov':{},
          'ob':{}}
for group, stuff in summary_dict.items():
    print(group)
    G=stuff['comm_graph']
    aes_dict[group]=an.aesthetics(G,15000,100, 'sans-serif', 'Bold', 'z_edge', (80,50), 1)

no
8
ov
9
ob
9


In [22]:
for key, graph in summary_dict.items():
    print(key)
    G=graph['comm_graph']
    aes=aes_dict[key]
    an.module_fig(G,'%s z-score'%key, basepath, aes)

no
8
8
[0.4375 1.3125 2.1875 3.0625 3.9375 4.8125 5.6875 6.5625]
ov
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]
ob
9
9
[0.44444444 1.33333333 2.22222222 3.11111111 4.         4.88888889
 5.77777778 6.66666667 7.55555556]


In [23]:
labels.set_index('Index', inplace=True)

In [24]:
lab_dict=labels.to_dict('index')

In [25]:
for group, mods in subgraph_dict.items():
    for mod, G in mods.items():
        nx.set_node_attributes(G,lab_dict, "labels")

In [26]:
g=subgraph_dict['no'][1]

In [27]:
nodes, color = zip(*nx.get_node_attributes(g, 'labels').items())

In [28]:
nx.get_node_attributes(g, 'labels')[1]['area']
# nx.get_node_attributes(g, 'labels').values()


'Default'

In [29]:
new={}
for x,y in nx.get_node_attributes(g, 'labels').items():
#     print(x)
    new[x]=y['area']

In [30]:
for x in color:
    print(x['area'])

Default
FrontoParietal
Default
Default
DorsalAttn
Default
FrontoParietal
None
FrontoParietal
VentralAttn
FrontoParietal
Cerebellum
FrontoParietal
Cerebellum
FrontoParietal
Cerebellum
Cerebellum
Cerebellum
Cerebellum
nan
Cerebellum
Cerebellum
Cerebellum
Cerebellum
nan
Cerebellum
Cerebellum


In [ ]:
groups = ['no','ov','ob']
mods = [2,4,0,5,2,6]
cc_=[11,16, 27, 35, 54, 58, 85]
for group in groups:
    for mod in mods:
        for x in cc_:
            if x in list(subgraph_dict[group][mod].nodes()):
                print('for %s in mod %i and node %i'%(group, mod, x))
                print(subgraph_dict[group][mod].nodes(data=True)[x])

In [31]:
def aesthetics(graph, node_size, font_size, font_family, font_weight, edge_att, plot_size, mod):
    aes={'general':{},
         'nodes':{},
         'edges':{}}
    # nodes
    color = np.array(list(graph.nodes))
    color = np.array(color)
    n_color=len(list(set(color)))
    print(n_color)
    aes['nodes']['color'] = color
    aes['nodes']['colormap'] = ['Set3', n_color]
    aes['nodes']['n_color'] = n_color
    aes['nodes']['max'] = float(color.max())
    aes['nodes']['min'] = float(color.min())
    aes['nodes']['font_size'] = int(font_size)
    aes['nodes']['font_family'] = font_family
    aes['nodes']['font_weight'] = font_weight
    aes['nodes']['node_size'] = int(node_size)
    #edges
    aes['edges']['colormap'] = plt.cm.gist_rainbow
    edges,weights = zip(*nx.get_edge_attributes(graph,edge_att).items())
    weights=np.array(weights)
    aes['edges']['width mod'] = mod
    aes['edges']['min'] = weights.min()
    aes['edges']['max'] = weights.max()
    # General
    aes['general']['plot_size']=plot_size #tuple 80,50
    return(aes)

In [32]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

def basic_graph(graph, scl, FONT, metric, size, tile, basepath, bmi, mod):
    #threshold
    e,w = zip(*nx.get_edge_attributes(graph, 'weight').items())
    purr=np.percentile(w, tile)
    g=an.threshold2(graph,purr)
    # style
    fig = plt.figure(figsize = size)
    ax1 = plt.subplot(111, aspect = 'equal')
    plt.subplots_adjust(left=0.25, bottom=0, right=0.75, top=1, wspace=0.2, hspace=0)

    positions=nx.circular_layout(g)
    
    # NODES
    nodes, color = zip(*nx.get_node_attributes(g, metric).items())
    nodes, label = zip(*nx.get_node_attributes(g, 'labels').items())
    nodes, size = zip(*nx.get_node_attributes(g, metric).items())
    new={}
    for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=y['area']
    
    #draws nodes
    color = np.array(color)
    colz=stats.zscore(color)
    nColormap=plt.cm.cool 
    cM=colz.max()
    cm=colz.min()

    scale=scl/colz.max()
    nx.draw_networkx_nodes(g,
                             positions,
                             node_color=colz,
                             node_size=np.square(colz)*scale,
                             alpha=0.8,
                             cmap= nColormap,
                             vmin=cm ,vmax=cM)

    #Styling for labels
    nx.draw_networkx_labels(g, 
                            positions,
                            new,
                            font_size=FONT,
                            font_family='sans-serif',
                            fontweight = 'bold',
                            horizontalalignment = 'center',
                            verticalalignment = 'top')


    # EDGES 
    edges,weights = zip(*nx.get_edge_attributes(g, 'weight').items())
    #draw edges
    weights=np.array(weights)
    eColormap=plt.cm.gist_rainbow #check here if you want different colors https://matplotlib.org/3.1.1/gallery/color/colormap_reference.html

    nx.draw_networkx_edges(g, 
                             positions,
                             edge_list=edges,
                             style='solid',
                             width = np.square(weights)*200,
                             edge_color = weights,
                             edge_cmap=eColormap,
                             edge_vmin=weights.min(),
                             edge_vmax=weights.max())
    
    #FInishing touches


    plt.axis('off')
    if bmi == 'no':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Average BMI'), fontsize = FONT)
    if bmi == 'ov':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'High BMI'), fontsize = FONT)
    if bmi == 'ob':
        plt.title("%s and edge weights of \n %s graph"%(metric, 'Very high BMI'), fontsize = FONT)
    

    axins1 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(0.15, 0., 1, 1), #(x0, y0, width, height)
                    bbox_transform=ax1.transAxes,
                    borderpad=1)
    
    axins2 = inset_axes(ax1,
                    width="2.5%",  # width = 50% of parent_bbox width
                    height="100%",  # height : 5%
                    loc='right',
                    bbox_to_anchor=(.25, 0, 1, 1),
                    bbox_transform=ax1.transAxes,
                    borderpad=1)

    sm = plt.cm.ScalarMappable(cmap=eColormap, norm=plt.Normalize(vmin = weights.min(), vmax=weights.max()))
    sm._A = []

    edge_bar=fig.colorbar(sm, cax=axins1, orientation="vertical")
    for l in edge_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    edge_bar.set_label('Strength of edge weight', fontsize = FONT)
    
    sn = plt.cm.ScalarMappable(cmap=nColormap, norm=plt.Normalize(vmin = cm, vmax=cM))
    sn._A = []
    
    rgb = sn.to_rgba(x=colz)
    save={}
    i = 0
    for x,y in nx.get_node_attributes(g, 'labels').items():
        save[x]=[y['IC'], y['area'], np.round(rgb[i]*255)]
        i=i+1
    
    node_bar=fig.colorbar(sn, cax=axins2, label='Module value')
    for l in node_bar.ax.yaxis.get_ticklabels():
        l.set_size(FONT)
    node_bar.set_label('%s'%metric,fontsize = FONT)
    
    
    basepath='/Users/gracer/Google Drive/HCP/HCP_graph/1200/images'
    if bmi == 'no':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Average BMI', mod)), format="PNG")
    if bmi == 'ov':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'High BMI', mod)), format="PNG")
    if bmi == 'ob':
        plt.savefig(os.path.join(basepath,"%s_%s_%s.png"%(metric, 'Very high BMI', mod)), format="PNG")
    plt.show()
    return(save)


In [33]:
basic_graph(subgraph_dict[group][mod], 50000, 50, 'PC', (75,50), 80, basepath, '%s'%group, '%i'%mod)

{97: ['IC_97', 'BrainStem', array([  0., 255., 255., 255.])],
 98: ['IC_98', 'VentralDiencephalon', array([188.,  67., 255., 255.])],
 99: ['IC_99', 'VentralDiencephalon', array([201.,  54., 255., 255.])],
 79: ['IC_79', 'BrainStem', array([245.,  10., 255., 255.])],
 85: ['IC_85', 'BrainStem', array([227.,  28., 255., 255.])],
 90: ['IC_90', 'BrainStem', array([114., 141., 255., 255.])],
 93: ['IC_93', 'BrainStem', array([255.,   0., 255., 255.])],
 94: ['IC_94', 'BrainStem', array([128., 127., 255., 255.])]}

In [38]:
wb_con = {
    'no':{2:{},5:{}},
    'ov': {2:{}, 5:{}},
    'ob':{2:{}, 6:{}}
}

for group, mods in wb_con.items():
    print(group)
    for mod, blank in mods.items():
        print(mod)  
        wb_con[group][mod]=basic_graph(subgraph_dict[group][mod], 50000, 50, 'PC', (75,50), 80, basepath, '%s'%group, '%i'%mod)

no
2
5
ov
2
5
ob
2
6


## Get the colors for workbench

In [35]:
for group, mods in wb_con.items():
    print(group)
    for mod, data in mods.items():
        print(mod)
        writepath = os.path.join(basepath,'%s_%i.txt'%(group, mod))
        f = open(writepath, "w")
        for key, value in data.items():
            print(key)
            x = value[2]
            print(x)
            f.write('LABEL_%i\n'%(key+1))
            f.write('%i %i %i %i %i\n'%(key+1, x[0], x[1], x[2], x[3]))
        f.close()

no
2
2
[163.  92. 255. 255.]
3
[  4. 251. 255. 255.]
7
[ 47. 208. 255. 255.]
10
[  0. 255. 255. 255.]
12
[ 36. 219. 255. 255.]
15
[ 71. 184. 255. 255.]
16
[ 16. 239. 255. 255.]
17
[255.   0. 255. 255.]
18
[179.  76. 255. 255.]
19
[102. 153. 255. 255.]
20
[ 83. 172. 255. 255.]
22
[112. 143. 255. 255.]
26
[120. 135. 255. 255.]
27
[ 18. 237. 255. 255.]
29
[ 48. 207. 255. 255.]
33
[109. 146. 255. 255.]
34
[108. 147. 255. 255.]
37
[125. 130. 255. 255.]
40
[ 83. 172. 255. 255.]
42
[ 86. 169. 255. 255.]
44
[ 37. 218. 255. 255.]
46
[ 57. 198. 255. 255.]
80
[105. 150. 255. 255.]
87
[191.  64. 255. 255.]
95
[  4. 251. 255. 255.]
96
[131. 124. 255. 255.]
5
64
[  0. 255. 255. 255.]
66
[243.  12. 255. 255.]
67
[208.  47. 255. 255.]
68
[123. 132. 255. 255.]
98
[133. 122. 255. 255.]
99
[148. 107. 255. 255.]
75
[ 46. 209. 255. 255.]
78
[134. 121. 255. 255.]
48
[255.   0. 255. 255.]
86
[160.  95. 255. 255.]
62
[156.  99. 255. 255.]
ov
0
0
[255.   0. 255. 255.]
24
[118. 137. 255. 255.]
38
[  0. 255. 255

## Get ICnames

In [ ]:
g = subgraph_dict['ob'][6]
new={}
for x,y in nx.get_node_attributes(g, 'labels').items():
        new[x]=[y['IC'], y['area']]

In [ ]:
new

In [ ]:
z={}
for group, dat in submod_dict.items():
    print(group)
    z[group]={}
    for module, data in dat.items():
        print(module)
        z[group][module]=data.join(labels)
no=pd.concat(list(z['no'].values()))
ov=pd.concat(list(z['ov'].values()))
ob=pd.concat(list(z['ob'].values()))
total=pd.concat([no,ov,ob])
total.rename(columns={'Unnamed: 2': 'ROI'}, inplace=True)
# new_df = total.merge(atlas[['X','Y','Z','ROI']], left_on='ROI', right_on = 'ROI', how='left')
# new_df.to_csv(os.path.join(basepath,'tmp','submodule_data.csv'), sep=',')


In [ ]:
total['modules'].describe()